In [10]:
import pandas as pd
import numpy as np
import pulp

In [11]:
def load_data(file_path):
    try:
        # Load flow matrix (w)
        w_df = pd.read_excel(file_path, sheet_name="w", header=0, index_col=0)
        print(w_df)
        # Convert to numpy array and fill NaN values with 0 if any
        w = w_df.fillna(0).to_numpy()
        w = np.array(w)
        
        # Get nodes - ensure they're consecutive integers starting from 1
        nodes = list(map(int, w_df.index.tolist()))
        n = len(nodes)
        
        # Load distance matrix (c)
        c_df = pd.read_excel(file_path, sheet_name="c", header=0, index_col=0)
        c = c_df.fillna(0).to_numpy()
        c = np.array(c)
        
        # Load fixed costs (f)
        f_df = pd.read_excel(file_path, sheet_name="f", header=None, index_col=0)
        f = f_df.iloc[:, 0].to_dict()
        
        # Calculate total outgoing flow (q_i) and incoming flow (d_j)
        # Using dictionary comprehension with proper indexing
        q = {node: sum(w[nodes.index(node)][nodes.index(j)] for j in nodes) for node in nodes}
        d = {node: sum(w[nodes.index(i)][nodes.index(node)] for i in nodes) for node in nodes}
        
        return w, c, f, nodes, q, d
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None, None, None, None

# Load the data
file_path = "Data assignment parcel transport 1 Large.xlsx"
w, c, f, nodes, q, d = load_data(file_path)

if nodes is None:
    print("Failed to load data. Please check the file path and format.")
else:
    print("Data loaded successfully!")
    print(f"Nodes: {nodes}")
    print(f"Outgoing flows (q): {q}")
    print(f"Incoming flows (d): {d}")
    print(f"Fixed costs (f): {f}")


# # Nodes (1-10)
nodes = [k for k in f.keys()]






    1   2   3   4   5   6   7   8   9   10  11  12  13   14  15
1    6  10   5   3   6   6   2   2   2   2   2   2   3    9   2
2   21  39  18  10  22  21   8   7   8   8   8   6  10   35   5
3    6  10   9   9   6   8   7   5   3   5   6   4   4   15   4
4    4   6  10  11   4   7   8   6   3   6   8   5   4   15   4
5    7  12   6   4   8   7   3   3   5   4   3   3   6   20   3
6   17  31  17  12  18  18   9   7   8   8   9   6  10   35   5
7    4   7  11  12   5   7   9   7   3   7   9   5   4   17   5
8    2   4   5   5   3   4   4   5   2   4   4   5   3   16   4
9    3   5   4   4   6   4   3   4   9   5   3   4  11   37   4
10   4   6   9   9   5   6   7   6   4   6   7   5   5   20   4
11   2   3   5   6   2   4   4   3   2   3   4   3   2    9   3
12   2   4   4   4   3   3   3   6   3   3   3   7   3   23   6
13   4   8   6   5   9   7   4   5  13   7   4   6  18   57   5
14  25  45  36  31  40  35  24  34  43  32  24  38  56  181  32
15  10  17  17  16  14  14  12  25  14  

In [12]:
# 4. Calculate Outgoing (q) and Incoming (d) Flows


q = {i: sum(w[i-1, :]) for i in nodes}  # Row sums (outgoing)
d = {j: sum(w[:, j-1]) for j in nodes}  # Column sums (incoming)


q,d




({1: 62,
  2: 226,
  3: 101,
  4: 101,
  5: 94,
  6: 210,
  7: 112,
  8: 70,
  9: 106,
  10: 103,
  11: 55,
  12: 77,
  13: 158,
  14: 676,
  15: 364},
 {1: 117,
  2: 207,
  3: 162,
  4: 141,
  5: 151,
  6: 151,
  7: 107,
  8: 125,
  9: 122,
  10: 115,
  11: 108,
  12: 130,
  13: 157,
  14: 610,
  15: 112})

In [13]:
# 2. Initiate the model

model = pulp.LpProblem("PostNL_Parcel_Optimization", pulp.LpMinimize)

In [14]:
# 3. Decision variables

# Binary variables
y = pulp.LpVariable.dicts("Hub", nodes, cat="Binary")  # y_k
x = pulp.LpVariable.dicts("Collection", [(i, k) for i in nodes for k in nodes], cat="Binary")  # x_ik
z = pulp.LpVariable.dicts("Distribution", [(l, j) for l in nodes for j in nodes], cat="Binary")  # z_lj

# Continuous variables
t = pulp.LpVariable.dicts("Transfer", [(k, l) for k in nodes for l in nodes], lowBound=0)  # t_kl

In [15]:
# 4. Objective function


# Cost multipliers
chi = 3  # Collection
alpha = 1  # Transfer
delta = 2  # Distribution

# Fixed costs
fixed_costs = pulp.lpSum(f[k] * y[k] for k in nodes)

# Collection costs (i -> k)
collection_costs = pulp.lpSum(chi * c[i-1][k-1] * q[i] * x[(i, k)] 
                              for i in nodes for k in nodes)

# Transfer costs (k -> l)
transfer_costs = pulp.lpSum(alpha * c[k-1][l-1] * t[(k, l)] 
                            for k in nodes for l in nodes)

# Distribution costs (l -> j)
distribution_costs = pulp.lpSum(delta * c[l-1][j-1] * d[j] * z[(l, j)] 
                               for l in nodes for j in nodes)

model += fixed_costs + collection_costs + transfer_costs + distribution_costs

In [16]:
# 5. Constraints


# Assignment constraints
for i in nodes:
    model += pulp.lpSum(x[(i, k)] for k in nodes) == 1  # Each non-hub assigns to one hub for collection

for j in nodes:
    model += pulp.lpSum(z[(l, j)] for l in nodes) == 1  # Each non-hub assigns to one hub for distribution

# Hub activation constraints
for i in nodes:
    for k in nodes:
        model += x[(i, k)] <= y[k]  # Can only assign to open hubs for collection

for l in nodes:
    for j in nodes:
        model += z[(l, j)] <= y[l]  # Can only assign to open hubs for distribution

# Flow conservation constraints
for k in nodes:
    model += pulp.lpSum(t[(k, l)] for l in nodes) == pulp.lpSum(q[i] * x[(i, k)] for i in nodes)  # Outflow = Inflow from non-hubs

for l in nodes:
    model += pulp.lpSum(t[(k, l)] for k in nodes) == pulp.lpSum(d[j] * z[(l, j)] for j in nodes)  # Inflow = Outflow to non-hubs

In [17]:

# 6. Getting the results and print

model.solve()

print("Status:", pulp.LpStatus[model.status])
print("Total Cost:", pulp.value(model.objective))

# Selected hubs
hubs = [k for k in nodes if pulp.value(y[k]) > 0.9]
print("\nSelected Hubs:", hubs)

# Collection assignments
print("\nCollection Assignments:")
for i in nodes:
    for k in nodes:
        if pulp.value(x[(i, k)]) > 0.9:
            print(f"Node {i} -> Hub {k} (Flow: {q[i]})")

# Distribution assignments
print("\nDistribution Assignments:")
for j in nodes:
    for l in nodes:
        if pulp.value(z[(l, j)]) > 0.9:
            print(f"Hub {l} -> Node {j} (Flow: {d[j]})")

# Inter-hub transfers
print("\nInter-Hub Transfers:")
for k in hubs:
    for l in hubs:
        flow = pulp.value(t[(k, l)])
        if flow > 0:
            print(f"Hub {k} -> Hub {l}: {flow:.1f} parcels")

Status: Optimal
Total Cost: 122694.0

Selected Hubs: [2, 6, 8, 14]

Collection Assignments:
Node 1 -> Hub 2 (Flow: 62)
Node 2 -> Hub 2 (Flow: 226)
Node 3 -> Hub 6 (Flow: 101)
Node 4 -> Hub 8 (Flow: 101)
Node 5 -> Hub 6 (Flow: 94)
Node 6 -> Hub 6 (Flow: 210)
Node 7 -> Hub 6 (Flow: 112)
Node 8 -> Hub 8 (Flow: 70)
Node 9 -> Hub 14 (Flow: 106)
Node 10 -> Hub 6 (Flow: 103)
Node 11 -> Hub 8 (Flow: 55)
Node 12 -> Hub 8 (Flow: 77)
Node 13 -> Hub 14 (Flow: 158)
Node 14 -> Hub 14 (Flow: 676)
Node 15 -> Hub 14 (Flow: 364)

Distribution Assignments:
Hub 2 -> Node 1 (Flow: 117)
Hub 2 -> Node 2 (Flow: 207)
Hub 6 -> Node 3 (Flow: 162)
Hub 8 -> Node 4 (Flow: 141)
Hub 6 -> Node 5 (Flow: 151)
Hub 6 -> Node 6 (Flow: 151)
Hub 6 -> Node 7 (Flow: 107)
Hub 8 -> Node 8 (Flow: 125)
Hub 14 -> Node 9 (Flow: 122)
Hub 14 -> Node 10 (Flow: 115)
Hub 14 -> Node 11 (Flow: 108)
Hub 8 -> Node 12 (Flow: 130)
Hub 14 -> Node 13 (Flow: 157)
Hub 14 -> Node 14 (Flow: 610)
Hub 14 -> Node 15 (Flow: 112)

Inter-Hub Transfers:
Hu